In [1]:
import requests
import json
import pandas as pd

In [2]:
json_files = []
pages = range(1,5) # This is because we already checked that there are 319 publications answering to this query
                   # the number 5 should be changed into the following number:
                   # (how_many_articles div 100) + 1 + 1
                   # This however implies querying the API twice. For this reason, since we do it once, 
                   # we just do it by hand
for pagenumber in pages:
    r = requests.get(f'https://doaj.org/api/search/articles/agent-based%20migration?page={pagenumber}&pageSize=100')
    json_file = json.loads(r.text)
    json_files.append(json_file)
json_files[0]

{'total': 319,
 'page': 1,
 'pageSize': 100,
 'timestamp': '2022-06-28T08:45:00.183663Z',
 'query': 'agent-based migration',
 'results': [{'last_updated': '2021-02-09T00:03:29Z',
   'bibjson': {'identifier': [{'id': '10.3390/molecules26040889',
      'type': 'doi'},
     {'id': '1420-3049', 'type': 'eissn'}],
    'journal': {'volume': '26',
     'number': '889',
     'country': 'CH',
     'issns': ['1420-3049'],
     'publisher': 'MDPI AG',
     'language': ['EN'],
     'title': 'Molecules'},
    'month': '02',
    'keywords': ['repurposing',
     'omarigliptin',
     'docking',
     'A2A adenosine receptor',
     'acetylcholine esterase receptor',
     'GLP-1'],
    'year': '2021',
    'start_page': '889',
    'subject': [{'code': 'QD241-441',
      'scheme': 'LCC',
      'term': 'Organic chemistry'}],
    'author': [{'affiliation': 'Pharmaceutical Chemistry Department, Faculty of Pharmacy, The British University in Egypt, El-Sherouk City, Cairo 11837, Egypt',
      'name': 'Bassam  M

In [3]:
print(json_file['results'][0]['bibjson'].keys())

how_many = json_file['total']
print(f'How many articles are we parsing: {how_many}')
count = 0
all_titles, all_abstracts, all_years, all_authors, all_affiliations, all_journals = [], [], [], [], [], []

for page in pages:
    current_json = json_files[page-1] # iterates from 1 to 4, so we do -1
    for i in range(len(current_json['results'])):
        count += 1
#         print(i)
        title      = current_json['results'][i]['bibjson']['title']
        abstract   = current_json['results'][i]['bibjson']['abstract']
        year       = current_json['results'][i]['bibjson']['year']
        journal    = current_json['results'][i]['bibjson']['journal']['title']
        authorlist = current_json['results'][i]['bibjson']['author']
        names, affiliations = [], []
        for author in authorlist:
            if 'name' in author.keys():
                names.append(author['name'])
            else: 
                names.append('NaN')
            if 'affiliation' in author.keys():
                affiliations.append(author['affiliation'])
            else:
                affiliations.append('NaN')
        names = ', '.join(names)
        affiliations = '; '.join(affiliations)
          
        all_titles.append(title)
        all_abstracts.append(abstract)
        all_years.append(year)
        all_authors.append(names)
        all_affiliations.append(affiliations)
        all_journals.append(journal)
        
print(f'\n{count} <- This number should be the same as the previous number')

dict_keys(['identifier', 'journal', 'month', 'end_page', 'keywords', 'year', 'start_page', 'subject', 'author', 'link', 'abstract', 'title'])
How many articles are we parsing: 319

319 <- This number should be the same as the previous number


In [4]:
df = {'authors':all_authors,
      'title':all_titles,
      'year':all_years,
      'abstract':all_abstracts,
      'affiliations':all_affiliations,
      'journal':all_journals
}

df = pd.DataFrame(df)
df.head()

,authors,title,year,abstract,affiliations,journal
0,"Bassam M. Ayoub, Haidy E. Michel, Shereen Mo...",Repurposing of Omarigliptin as a Neuroprotecti...,2021,The authors in the current work suggested the ...,"Pharmaceutical Chemistry Department, Faculty o...",Molecules
1,"Willy Brodus Uwan, Ari Fahrial Syam, C Rinaldi...",The Difference in Prevalence of Helicobacter p...,2016,Introduction. Helicobacter pylori (H. pylori) ...,Bagian Penyakit Dalam Rumah Sakit Santo Antoni...,Jurnal Penyakit Dalam Indonesia
2,"Xinli Wang, Liye Wang, Lijun Xie, Zuoxu Xie, L...",Design and Synthesis of a Novel NIR Celecoxib-...,2020,Cyclooxygenase-2 (COX-2) imaging agents are po...,"Department of Medical Oncology, Fujian Medical...",Molecules
3,"Mikio Saito, Manabu Abe, Tomoyasu Furukawa, Mo...",Study on Patients Who Underwent Suspected Diag...,2014,Background:: There are problems in diagnosis o...,"Laboratory of Clinical Pharmacy, Faculty of Ph...",Allergology International
4,"Zhiwei Teng, Bingtao Wang, Yingying Hu, Wei Zh...",High performance ultrafiltration membrane by c...,2021,Ultrafiltration (UF) membrane technology has d...,State Key Laboratory Breeding Base of Green Ch...,Polymer Testing


In [5]:
df.to_csv('20220628DOAJ.csv')